In [1]:
# Necessary imports\n",
import os
import pandas as pd
import numpy as np

In [2]:
# Get the COVID dataset
filename1 = os.path.join(os.getcwd(), "weekly_covid_county_level_US.csv")
df1 = pd.read_csv(filename1, header=0)

# Get the weather dataset
filename2 = os.path.join(os.getcwd(), "weekly_tavg_county_cont_US.csv")
df2 = pd.read_csv(filename2, header=0)

In [4]:
# Rename county_name to county in weekly_covid_county_level_US.csv
df1.rename(columns={'county_name': 'County'}, inplace=True)
df1.rename(columns={'report_date': 'Week'}, inplace=True)
df1.rename(columns={'state_name': 'State'}, inplace=True)

# Sort df1 by County to match with df2
sorted_df1 = df1.sort_values(by=['State','County', 'Week'])
#display(sorted_df1)

# Sort df2 by County to match with df1
sorted_df2 = df2.sort_values(by=['State','County', 'Week'])
    
#Drop 10/06 and 10/13 and 05/28/23 - 06/06/24 to match with when COVID data ends from df2 
values_todrop = ['2022-10-06', '2022-10-13', '2023-05-18', '2023-05-25', '2023-06-01', '2023-06-08', '2023-06-15', '2023-06-22', '2023-06-29', '2023-07-06',
'2023-07-13', '2023-07-20', '2023-07-27', '2023-08-03', '2023-08-10', '2023-08-17', '2023-08-24', '2023-08-31', '2023-09-07', '2023-09-14', '2023-09-21',
'2023-09-28', '2023-10-05', '2023-10-12', '2023-10-19', '2023-10-26', '2023-11-02', '2023-11-09', '2023-11-16', '2023-11-23', '2023-11-30', '2023-12-07',
'2023-12-14', '2023-12-21', '2023-12-28', '2024-01-04', '2024-01-11', '2024-01-18', '2024-01-25', '2024-02-01', '2024-02-08', '2024-02-15', '2024-02-22',
'2024-02-29', '2024-03-07', '2024-03-14', '2024-03-21', '2024-03-28', '2024-04-04', '2024-04-11', '2024-04-18', '2024-04-25', '2024-05-02', '2024-05-09',
'2024-05-16', '2024-05-23', '2024-05-30', '2024-06-06']

sorted_df2 = sorted_df2[~sorted_df2['Week'].isin(values_todrop)]
#display(sorted_df2)
#print(sorted_df2.head(250))

In [8]:
# Merge the two datasets into a new dataset on the county column
# The new df is sorted by the County column
df = pd.merge(sorted_df2, sorted_df1, on=['County', 'Week'], how='outer')
#display(df)
   
# Drop FIPS, fips_code and state_abbr columns since they are unnecessary and repeating
df = df.drop(columns=['FIPS', 'fips_code', 'state_abbr'])

print(df.head(130))
#display(df)

    State_x     County        Week  Temperature         State_y  \
0        SC  Abbeville  2022-10-20        14.22  South Carolina   
1        SC  Abbeville  2022-10-27        13.56  South Carolina   
2        SC  Abbeville  2022-11-03        15.76  South Carolina   
3        SC  Abbeville  2022-11-10        18.69  South Carolina   
4        SC  Abbeville  2022-11-17        10.72  South Carolina   
..      ...        ...         ...          ...             ...   
125      KY      Adair  2022-10-20         9.96        Kentucky   
126      KY      Adair  2022-10-20         9.96        Missouri   
127      KY      Adair  2022-10-20         9.96        Oklahoma   
128      MO      Adair  2022-10-20         5.62            Iowa   
129      MO      Adair  2022-10-20         5.62        Kentucky   

     cases_per_100K_7_day_count_change  \
0                                40.77   
1                                16.31   
2                                44.85   
3                          